In [1]:
import numpy as np
import pandas as pd

In [2]:
import scipy.io

In [3]:
data = scipy.io.loadmat('/Users/Sophie/Library/CloudStorage/OneDrive-Personnel/IP/Data Stream Processing/PySAD_to_River/data/arrhythmia.mat') 

In [4]:
X = data['X']
y = data['y']

In [5]:
from anomaly.base import AnomalyDetector

Import our model

In [12]:
from models.xstream import xStream

NameError: name 'BaseTransformer' is not defined